In [2]:
# 📦 DeepLabV3 Version (complète)
!pip install segmentation-models-pytorch albumentations opencv-python tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 4.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found exis

In [3]:
import os
import random
import numpy as np
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm

def train_deeplabv3_model(
    data_root,
    image_size=(256, 256),
    encoder="resnet34",
    encoder_weights="imagenet",
    batch_size=8,
    epochs=25,
    lr=1e-4,
    save_path="deeplabv3_model.pth",
    seed=42
):
    random.seed(seed)
    torch.manual_seed(seed)

    image_dir = os.path.join(data_root, "images")
    mask_dir = os.path.join(data_root, "masks")
    all_images = sorted([f for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])

    train_imgs, temp_imgs = train_test_split(all_images, test_size=0.3, random_state=seed)
    val_imgs, test_imgs = train_test_split(temp_imgs, test_size=1/3, random_state=seed)

    class MedicalDataset(Dataset):
        def __init__(self, files, image_dir, mask_dir, transform=None):
            self.files = files
            self.image_dir = image_dir
            self.mask_dir = mask_dir
            self.transform = transform

        def __len__(self):
            return len(self.files)

        def __getitem__(self, idx):
            img_name = self.files[idx]
            img_path = os.path.join(self.image_dir, img_name)
            base_name, ext = os.path.splitext(img_name)
            mask_name = base_name + "_mask" + ext
            mask_path = os.path.join(self.mask_dir, mask_name)
            if not os.path.exists(mask_path):
                mask_name = base_name + ext
            mask_path = os.path.join(self.mask_dir, mask_name)

            image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            image = np.stack([image] * 3, axis=-1)

            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            mask = (mask > 127).astype(np.float32)

            if self.transform:
                augmented = self.transform(image=image, mask=mask)
                image = augmented["image"]
                mask = augmented["mask"]

            return image, mask.unsqueeze(0).float()

    transform = A.Compose([
        A.Resize(*image_size),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),
        A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
        ToTensorV2()
    ])

    train_ds = MedicalDataset(train_imgs, image_dir, mask_dir, transform)
    val_ds = MedicalDataset(val_imgs, image_dir, mask_dir, transform)
    test_ds = MedicalDataset(test_imgs, image_dir, mask_dir, transform)

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_ds, batch_size=1, shuffle=False)

    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = smp.DeepLabV3(
        encoder_name=encoder,
        encoder_weights=encoder_weights,
        in_channels=3,
        classes=1,
        activation=None
    ).to(DEVICE)

    if torch.cuda.device_count() > 1:
        model = torch.nn.DataParallel(model)

    dice_loss = smp.losses.DiceLoss(mode='binary')
    bce_loss = smp.losses.SoftBCEWithLogitsLoss()

    def combined_loss(preds, targets):
        return dice_loss(torch.sigmoid(preds), targets) + bce_loss(preds, targets)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        loop = tqdm(train_loader, desc=f"[Train] Epoch {epoch+1}/{epochs}")

        for images, masks in loop:
            images, masks = images.to(DEVICE), masks.to(DEVICE)
            preds = model(images)
            loss = combined_loss(preds, masks)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            loop.set_postfix(loss=loss.item())

        print(f"✅ Epoch {epoch+1} — Train Loss: {total_loss/len(train_loader):.4f}")

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for images, masks in val_loader:
                images, masks = images.to(DEVICE), masks.to(DEVICE)
                preds = model(images)
                loss = combined_loss(preds, masks)
                val_loss += loss.item()
        print(f"🔍 Epoch {epoch+1} — Val Loss: {val_loss/len(val_loader):.4f}")

    torch.save(model.state_dict(), save_path)
    print(f"📦 Modèle sauvegardé : {save_path}")
    return model, test_loader
    


/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.8' (you have '2.0.5'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [4]:
# Utilisation DeepLabV3
model, test_loader = train_deeplabv3_model(
    data_root="/kaggle/input/brain-breast-min-data-segmentation/data",
    image_size=(512, 512),
    encoder="resnet34",
    encoder_weights="imagenet",
    batch_size=32,  # ✅ ne pas mettre 1 ici
    epochs=25,
    lr=1e-4,
    save_path="deeplabv3_model.pth",
    seed=42
)


/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:87: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/87.3M [00:00<?, ?B/s]

[Train] Epoch 1/25: 100%|██████████| 29/29 [01:51<00:00,  3.84s/it, loss=1.15]


✅ Epoch 1 — Train Loss: 1.3501
🔍 Epoch 1 — Val Loss: 1.7710


...

✅ Epoch 23 — Train Loss: 0.9302
🔍 Epoch 23 — Val Loss: 0.9450


[Train] Epoch 24/25: 100%|██████████| 29/29 [01:45<00:00,  3.63s/it, loss=0.937]


✅ Epoch 24 — Train Loss: 0.9228
🔍 Epoch 24 — Val Loss: 0.9446


[Train] Epoch 25/25: 100%|██████████| 29/29 [01:45<00:00,  3.63s/it, loss=0.939]


✅ Epoch 25 — Train Loss: 0.9192
🔍 Epoch 25 — Val Loss: 0.9358
📦 Modèle sauvegardé : deeplabv3_model.pth


In [6]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import torch

def evaluate_model(model, test_loader, device, threshold=0.5):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, masks in test_loader:
            images = images.to(device)
            masks = masks.to(device)

            outputs = torch.sigmoid(model(images))

            preds = (outputs.cpu().numpy() > threshold).astype(int).flatten()
            labels = (masks.cpu().numpy() > 0).astype(int).flatten()  # Binarisation

            all_preds.extend(preds)
            all_labels.extend(labels)

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='binary')
    precision = precision_score(all_labels, all_preds, average='binary')
    recall = recall_score(all_labels, all_preds, average='binary')

    print(f"Accuracy  : {accuracy:.4f}")
    print(f"F1 Score  : {f1:.4f}")
    print(f"Precision : {precision:.4f}")
    print(f"Recall    : {recall:.4f}")

# Appel après entraînement
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
evaluate_model(model, test_loader, DEVICE)


Accuracy  : 0.9747
F1 Score  : 0.7710
Precision : 0.8099
Recall    : 0.7356
